In [27]:
import os
import pandas as pd
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.agents import load_tools, initialize_agent, AgentType
from getpass import getpass

In [28]:
os.environ['OPENAI_API_KEY'] = getpass("OpenAI Key:")